In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 26.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 514.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 23.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 540.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 29.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 1.4 MB/s eta 0:00:0000:010

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=22a5e1dc8606eae7f43e492e7a58d6d9fab56a2d37d3f9bded31bda66e67260e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0a/f5/dd/9d00836c4e9e279c2a59d5b0ab72dafa66cbc626a327c550dd
  Created wheel for lit: filename=lit-16.0.6-py3-none-any.whl size=93582 sha256=9d652aa98ce2d22a7febee1c4df7e61820d7a666b6bb9db7aec68fbb91a504ea
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1a/56/14/294a6c208bce35b0fc3170fe1049b2fd3f61ce6495fc3870b3
Successfully built sentence-transformers lit


In [102]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.estimator import Estimator
from sagemaker.session import s3_input
from sagemaker import KMeans
import boto3
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.decomposition import TruncatedSVD
import os
import json

In [3]:
# SageMaker 실행 역할 가져오기
role = get_execution_role()

# SageMaker 세션 생성
sagemaker_session = sagemaker.Session()


In [65]:
# S3에 액세스할 수 있는 클라이언트 생성
s3 = boto3.client('s3')
bucket = 'dothis-ai-test'
data_key = 'data/video_data_20230414.txt'

# S3에서 파일 가져오기
response = s3.get_object(Bucket=bucket, Key=data_key)

# 파일 내용 읽기
file_content = response['Body'].read().decode('utf-8')

# 파일 내용 출력
train_data = file_content[1:].split("\r\n")[:200]
test_data = file_content[1:].split("\r\n")[200:300]

In [ ]:
# SentenceTransformer을 사용한 문장 임베딩
embedding_name = "jhgan"
sentence_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [66]:
# vectors = [sentence_model.encode(vector) for vector in tqdm(train_data)]
vectors = [sentence_model.encode(vector) for vector in train_data]
sentence_vectors = np.vstack(vectors)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jhgan/ko-sroberta-multitask
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
# lsa로 차원축소

dim_name = "lsa"
n_components = 100
svd = TruncatedSVD(n_components=n_components)
train_lsa_sentence_vectors = svd.fit_transform(sentence_vectors)
train_lsa_sentence_vectors.shape

(200, 100)

In [96]:
# K-Means estimator 생성

kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.m5.xlarge',
                output_path='s3://{}/models'.format(bucket),
                k=10,
               feature_dim=n_components,
               base_job_name='kmeans-test-1')

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [97]:
# K-Means 모델 훈련

# 학습 작업 실행
kmeans.fit(kmeans.record_set(train_lsa_sentence_vectors))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-test-1-2023-06-15-07-19-22-147


2023-06-15 07:19:24 Starting - Starting the training job...
2023-06-15 07:19:39 Starting - Preparing the instances for training...
2023-06-15 07:20:31 Downloading - Downloading input data...
2023-06-15 07:20:51 Training - Downloading the training image......
2023-06-15 07:21:51 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/15/2023 07:22:03 INFO 140230851786560] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_to_read': 'false', '_enable_profiler': 'false', '_kvstore': 'auto', '_lo


2023-06-15 07:22:22 Uploading - Uploading generated training model
2023-06-15 07:22:22 Completed - Training job completed
Training seconds: 112
Billable seconds: 112


In [99]:
# 추론 엔드포인트 생성
endpoint_name = 'kmeans-test-1'  # SageMaker 엔드포인트 이름

kmeans.deploy(initial_instance_count=1, 
              instance_type='ml.m5.xlarge',
              endpoint_name=endpoint_name)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: kmeans-test-1-2023-06-15-07-23-02-662
INFO:sagemaker:Creating endpoint-config with name kmeans-test-1
INFO:sagemaker:Creating endpoint with name kmeans-test-1


-----!

In [70]:
# 추론 수행

# vectors = [sentence_model.encode(vector) for vector in tqdm(test_data)]
vectors = [sentence_model.encode(vector) for vector in test_data]

sentence_vectors = np.vstack(vectors)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
n_components = 100
svd = TruncatedSVD(n_components=n_components)
test_lsa_sentence_vectors = svd.fit_transform(sentence_vectors)
test_lsa_sentence_vectors.shape

(100, 100)

In [116]:
# 입력 데이터를 CSV 형식으로 변환
input_data = pd.DataFrame(test_lsa_sentence_vectors).astype('float32').to_csv(header=False, index=False)

'7.3241773,-1.8803644,-0.41338643,-2.4825091,-0.16728756,-0.11367648,-0.20906824,0.33585262,-0.258572,-0.27967912,0.164852,-3.0562007,1.6303061,0.67225087,1.0634077,-0.6326853,0.8240596,1.1413313,-0.7644789,-0.7818035,-0.61597735,1.1892904,-0.6788063,-0.065096654,1.1187471,0.40876895,-0.78082305,-0.31056464,-1.2350929,0.121704996,0.6660565,0.43819004,0.07447849,0.75947404,-0.5940561,-0.85058945,0.4201797,-0.17102188,-0.81861794,1.3961356,-0.8763545,-0.90311885,-0.18704362,0.6047842,-0.26248395,0.3936023,-0.5862213,0.0811246,-0.88385594,0.19508834,0.48352748,-1.0606055,-0.21412392,-0.4545701,-0.21253635,1.5293517,0.111537084,-0.1926604,-0.24558862,0.7821809,-0.22580576,-0.9253751,-0.13124138,0.32152098,0.42657548,0.28310013,0.18145995,-0.49,0.13268894,0.113314316,-0.24723354,-0.36260766,-0.4324189,0.999928,0.16700456,-0.3652249,0.20846796,0.11328077,0.07868861,-0.70663,-0.095877156,1.5409133,-0.83478844,1.2781451,0.48226136,-0.31030422,0.12919551,0.75872564,-0.33996472,1.137219,0.254411

In [138]:
# 추론 요청
runtime_client = boto3.client('sagemaker-runtime')
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=input_data)


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint kmeans-test-1 of account 118709199903 not found.

In [137]:
# 추론 결과 파싱
import ast


result = response['Body'].read().decode('utf-8')

# 문자열을 딕셔너리로 변환
dictionary = ast.literal_eval(result)
result = dictionary["predictions"]

# 클러스터링 결과 출력
print(f'Input data: {data}')
print(f'Cluster labels: {result}')


SyntaxError: invalid syntax (<unknown>, line 0)

In [136]:
# SageMaker 클라이언트 생성
sagemaker_client = boto3.client('sagemaker')

# 엔드포인트 삭제 요청
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '62535589-ef4f-45f5-8bca-6be091c6d711',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '62535589-ef4f-45f5-8bca-6be091c6d711',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 15 Jun 2023 08:21:02 GMT'},
  'RetryAttempts': 0}}